In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


data = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")

deleted_columns = ['CAEC','SMOKE','CH2O','TUE','CALC','SCC']

data = data.drop(deleted_columns, axis=1)

columns_to_encode = ['Gender', 'family_history_with_overweight', 'FAVC', 'MTRANS']

# Select the columns to encode
df_to_encode = data[columns_to_encode]

other_columns = data.drop(columns_to_encode, axis=1)

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the encoded DataFrame
encoded_array = encoder.fit_transform(df_to_encode)

# Convert the encoded array back to a DataFrame
df_encoded_onehot = pd.DataFrame(encoded_array.toarray(), columns=encoder.get_feature_names_out(df_to_encode.columns))

df_final = pd.concat([other_columns, df_encoded_onehot], axis=1)

column_to_move = df_final.pop('NObeyesdad')

# Reinsert the column at the end
df_final['NObeyesdad'] = column_to_move

# Custom mapping for target class (obesity level)
feature_mapping = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Overweight_Level_I': 2,
    'Overweight_Level_II': 3,
    'Obesity_Type_I': 4,
    'Obesity_Type_II': 5,
    'Obesity_Type_III': 6,
}

df_final['NObeyesdad'] = df_final['NObeyesdad'].map(feature_mapping)

X = df_final.drop('NObeyesdad', axis=1)     # Features
y = df_final['NObeyesdad']                  # Target variable

# split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=462)

# min-max normalization
min_vals = X_train.min()
max_vals = X_train.max()

column_to_be_normalized = ['Age','Height','Weight','FCVC','NCP','FAF']

for col in X_train:
    X_test[col] = (X_test[col] - min_vals[col]) / (max_vals[col] - min_vals[col])
    X_train[col] = (X_train[col] - min_vals[col]) / (max_vals[col] - min_vals[col])

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# converting training and test values into tensors
X_train_tensor = torch.tensor(X_train.values.astype(np.float32))
X_test_tensor = torch.tensor(X_test.values.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

class ArtificialNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ArtificialNeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        layer_sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        return self.layers[-1](x)   # No activation function applied to the output layer

input_size = X_train_tensor.shape[1]
hidden_sizes = [64,64,64]           # hidden layer size is hyperparameter
output_size = len(y.unique())

model = ArtificialNeuralNetwork(input_size, hidden_sizes, output_size)

criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001) # learning rate is hyperparameter

num_epochs = 100

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the accuracy of the model on the test set
with torch.no_grad():
    model.eval()
    outputs = model.forward(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print(f"\nTest Accuracy: {accuracy:.4f}")

Epoch [1/100], Loss: 1.8864
Epoch [2/100], Loss: 1.6336
Epoch [3/100], Loss: 1.3778
Epoch [4/100], Loss: 1.1890
Epoch [5/100], Loss: 1.1442
Epoch [6/100], Loss: 0.8943
Epoch [7/100], Loss: 0.7421
Epoch [8/100], Loss: 0.7521
Epoch [9/100], Loss: 0.5015
Epoch [10/100], Loss: 0.4430
Epoch [11/100], Loss: 0.6660
Epoch [12/100], Loss: 0.4362
Epoch [13/100], Loss: 0.3443
Epoch [14/100], Loss: 0.2709
Epoch [15/100], Loss: 0.3259
Epoch [16/100], Loss: 0.3065
Epoch [17/100], Loss: 0.1663
Epoch [18/100], Loss: 0.2136
Epoch [19/100], Loss: 0.2095
Epoch [20/100], Loss: 0.1988
Epoch [21/100], Loss: 0.1579
Epoch [22/100], Loss: 0.4340
Epoch [23/100], Loss: 0.2445
Epoch [24/100], Loss: 0.1409
Epoch [25/100], Loss: 0.1631
Epoch [26/100], Loss: 0.2141
Epoch [27/100], Loss: 0.1083
Epoch [28/100], Loss: 0.1420
Epoch [29/100], Loss: 0.2369
Epoch [30/100], Loss: 0.3873
Epoch [31/100], Loss: 0.0829
Epoch [32/100], Loss: 0.2567
Epoch [33/100], Loss: 0.2301
Epoch [34/100], Loss: 0.1675
Epoch [35/100], Loss: 0